<a href="https://colab.research.google.com/github/seanreed1111/colab-demos/blob/master/GRADIO_file_editor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Using Gradio to create a simple interface.

Check out the library on [github](https://github.com/gradio-app/gradio-UI) and see the [getting started](https://gradio.app/getting_started.html) page for more demos.

We'll start with a basic function that greets an input name.

In [ ]:
def greet(name):
  return "Hello " + name + "!"

greet("World")

'Hello World!'

Now we'll wrap this function with a Gradio interface.

In [ ]:
!pip install -qqq gradio loguru pyyaml

In [ ]:
import gradio as gr
import os
import json
import yaml

def read_file(file_path):
    _, ext = os.path.splitext(file_path)
    with open(file_path, 'r') as f:
        content = f.read()

    if ext == ".json":
        try:
            json_content = json.loads(content)
            yaml_content = yaml.dump(json_content, default_flow_style=False)
            return yaml_content, True, "yaml"
        except json.JSONDecodeError:
            return "Invalid JSON file!", False, "yaml"

    return content, False, "text"

def convert_content(content, current_format):
    if current_format == "yaml":
        try:
            json_content = yaml.safe_load(content)
            return json.dumps(json_content, indent=4), "json"
        except yaml.YAMLError:
            return "Invalid YAML content!", "yaml"
    else:
        try:
            json_content = json.loads(content)
            return yaml.dump(json_content, default_flow_style=False), "yaml"
        except json.JSONDecodeError:
            return "Invalid JSON content!", "json"

def save_file(content, file_path, directory, is_json, current_format):
    if not os.path.isdir(directory):
        return "Invalid directory!"

    file_name = os.path.basename(file_path)
    save_path = os.path.join(directory, file_name)

    if is_json:
        if current_format == "yaml":
            try:
                json_content = yaml.safe_load(content)
                with open(save_path, 'w') as f:
                    json.dump(json_content, f, indent=4)
            except yaml.YAMLError:
                return "Invalid YAML content!"
        else:
            try:
                json_content = json.loads(content)
                with open(save_path, 'w') as f:
                    json.dump(json_content, f, indent=4)
            except json.JSONDecodeError:
                return "Invalid JSON content!"
    else:
        with open(save_path, 'w') as f:
            f.write(content)

    return f"File saved successfully at {save_path}!"

def main():
    with gr.Blocks() as app:
        gr.Markdown("## File Editor App with JSON/YAML Conversion")

        file_input = gr.File(label="Upload a file", type="filepath")
        directory_input = gr.Textbox(label="Directory to Save", placeholder="Enter directory path")
        text_area = gr.Code(label="File Content", language="yaml", lines=20)
        toggle_button = gr.Button("Toggle JSON/YAML")
        save_button = gr.Button("Save Changes")
        save_status = gr.Textbox(label="Status", interactive=False)
        is_json_state = gr.State()
        current_format_state = gr.State()

        def update_editor(file_path):
            content, is_json, current_format = read_file(file_path)
            return content, is_json, current_format

        file_input.change(update_editor, inputs=file_input, outputs=[text_area, is_json_state, current_format_state])
        toggle_button.click(convert_content, inputs=[text_area, current_format_state], outputs=[text_area, current_format_state])
        save_button.click(save_file, inputs=[text_area, file_input, directory_input, is_json_state, current_format_state], outputs=save_status)

    app.launch()

if __name__ == "__main__":
    main()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5739b2e535195811b9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


That's all! Go ahead and open that share link in a new tab. Check out our [getting started](https://gradio.app/getting_started.html) page for more complicated demos.